## Post-deployment QA: Reply tool v2.0 instrumentation (DB) [T252930](https://phabricator.wikimedia.org/T252930)

## Part 2 :  Add fields to VisualEditorFeatureUse

This task is about ensuring that the newly added events and fields are being logged correctly in database.

Test scenarios and status are documented in this [QC document](https://docs.google.com/spreadsheets/d/1txtypYKZHCiZEFnkye9LX7riF3VEippMmvN6ILs_BUQ/edit#gid=605234531)

**Observations:**

- Overall, data for new fields added in VisualEditorFeatureUse looks good
- Looked at data from 07-01-2020 and found a huge number of Null 'editor_Interface' records for Mobile platform. This bug was identified during Instrumentation QA in [T252925](https://phabricator.wikimedia.org/T252925) Issue #1
- Verified that these null records have been reducing since the fix applied on Aug 4, 2020 and assigned a valid editor_interface. 
- Observed a few sessions in VisualEditorFeatureUse schema (that have null editor_interface) that do not exist in EditAttempStep. 
We are not sure of the exact cause of this, but this may not be an issue as the number of events with null editor_interface is extremely low from Aug 4, 2020 onwards (almost 0 now).

In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import numpy as np
import pandas as pd
from tabulate import tabulate
from wmfdata import charting, hive, mariadb
from wmfdata.charting import comma_fmt, pct_fmt
from wmfdata.utils import df_to_remarkup, pct_str
import matplotlib.ticker as ticker
@ticker.FuncFormatter
def million_formatter(x, pos):
    return "%d M" % round(x/1000000)
@ticker.FuncFormatter
def thousand_formatter(x, pos):
    return "%d K" % round(x/1000)


## Schema: VisualEditorFeatureUse
For all the Fields added in [T252924](https://phabricator.wikimedia.org/T252924)  
New fields added to the schema:
- Integration
- Editor_interface
- Platform
- User_id
- User_editcount
- Bucket

In [2]:
# Count of all events in visualeditorfeatureuse with Integration=discussiontools

count_vefu_integ_edinterface = hive.run("""
SELECT
  event.platform AS platform,
  event.integration AS integration,
  event.editor_interface AS editor_interface,
  COUNT(*) AS events
FROM event.visualeditorfeatureuse
WHERE
  year = 2020 
  AND month>=7
  AND wiki <> 'testwiki'
GROUP BY --date_format(dt, "yyyy-MM-dd"), 
event.platform, event.integration, event.editor_interface
""")

### Count of Events by Platform, Integration and editor interface

In [3]:
count_vefu_integ_edinterface.sort_values('integration')

,platform,integration,editor_interface,events
5,desktop,discussiontools,visualeditor,2662
6,desktop,discussiontools,wikitext,139
1,desktop,page,wikitext,22127
2,phone,page,NaN,4480345
3,phone,page,visualeditor,3805802
4,desktop,page,visualeditor,4636063
7,desktop,page,wikitext-2017,206892
8,other,page,visualeditor,3375
9,phone,page,wikitext,301607
0,NaN,NaN,NaN,28


**We see a huge number of Null 'editor_Interface' for Mobile platform.**   
This bug was identified in [T252925](https://phabricator.wikimedia.org/T252925) Issue #1

In [7]:
# Count of all events in visualeditorfeatureuse with Integration=discussiontools

count_vefu_null_edint_day = hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.integration AS integration,
  event.editor_interface AS editor_interface,
  event.platform AS platform,
  COUNT(*) AS events
FROM event.visualeditorfeatureuse
WHERE
  year = 2020 
  AND month>=8
  AND wiki <> 'testwiki'
  AND event.editor_interface is null --filter for issue records
  AND event.integration='page'
GROUP BY date_format(dt, "yyyy-MM-dd"), 
event.integration, event.editor_interface, event.platform
ORDER BY date DESC LIMIT 50
""")

**This seems to have been reducing and almost fixed from August 4th onwards.**

In [8]:
count_vefu_null_edint_day

,date,integration,editor_interface,platform,events
0,2020-09-11,page,NaN,phone,1
1,2020-09-05,page,NaN,phone,7
2,2020-08-26,page,NaN,phone,1
3,2020-08-22,page,NaN,phone,1
4,2020-08-16,page,NaN,phone,2
5,2020-08-15,page,NaN,phone,3
6,2020-08-14,page,NaN,phone,8
7,2020-08-13,page,NaN,phone,14
8,2020-08-12,page,NaN,phone,12
9,2020-08-11,page,NaN,phone,1


In [9]:
# Count of all events in visualeditorfeatureuse with Integration=discussiontools

count_vefu_edint_phone = hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.platform AS platform,
  event.editor_interface AS editor_interface,
  COUNT(*) AS events
FROM event.visualeditorfeatureuse
WHERE
  year = 2020 
  AND month>=7
  AND wiki <> 'testwiki'
 -- AND event.integration='page'
  AND event.platform="phone"
  AND day IN (1,3,4,10,16,26) -- limiting the number of days for a quick view
GROUP BY date_format(dt, "yyyy-MM-dd"), 
event.integration, event.editor_interface, event.platform
""")

Verified if the null editor_interface records for Mobile platform have now been assigned a valid editor_interface.  
We see that there is an increase in the visualeditor/wikitext editor values for Mobile Platform and a drop in the null values

In [10]:
count_vefu_edint_phone.pivot(index='date', columns='editor_interface').fillna(0)

platform                          events               \
editor_interface      NaN visualeditor wikitext       NaN visualeditor   
date                                                                     
2020-07-01          phone            0        0  133905.0          0.0   
2020-07-03          phone            0        0  132844.0          0.0   
2020-07-04          phone            0        0  133378.0          0.0   
2020-07-10          phone            0        0  132912.0          0.0   
2020-07-16          phone            0        0  132120.0          0.0   
2020-07-26          phone            0        0  135634.0          0.0   
2020-08-01          phone        phone    phone  135026.0       4874.0   
2020-08-03          phone        phone    phone   65479.0      67506.0   
2020-08-04          phone        phone    phone      57.0     130025.0   
2020-08-10          phone        phone    phone       2.0     126080.0   
2020-08-16          phone        phone    phone       2.0     132135.0   
2020-08-26          phone        phone    phone       1.0     138270.0   
2020-09-01              0        phone    phone       0.0     125113.0   
2020-09-03              0        phone    phone       0.0     116356.0   
2020-09-04              0        phone    phone       0.0     118594.0   
2020-09-10              0        phone    phone       0.0     115495.0   

                           
editor_interface wikitext  
date                       
2020-07-01            0.0  
2020-07-03            0.0  
2020-07-04            0.0  
2020-07-10            0.0  
2020-07-16            0.0  
2020-07-26            0.0  
2020-08-01          845.0  
2020-08-03         5296.0  
2020-08-04         9879.0  
2020-08-10        10382.0  
2020-08-16        10255.0  
2020-08-26        10431.0  
2020-09-01         9818.0  
2020-09-03         9978.0  
2020-09-04         9905.0  
2020-09-10        10279.0

In [8]:
# Count of all events in visualeditorfeatureuse 

count_vefu_userid = hive.run("""
SELECT
  event.integration AS integration,
  event.editor_interface AS editor_interface,
  event.platform AS platform,
  event.bucket AS bucket,
  sum(cast(event.user_id!=0 as int)) as logged_in_users,
  sum(cast(event.user_id=0 as int)) as logged_out_users
FROM event.visualeditorfeatureuse
WHERE
  year = 2020 
  AND month>=7
  AND wiki <> 'testwiki'
GROUP BY
event.integration, event.editor_interface, event.platform, event.bucket
""")

### User_id check : Logged In vs Logged Out users for the new fields added to VEfu

In [9]:
count_vefu_userid.sort_values('integration').fillna(0)

,integration,editor_interface,platform,bucket,logged_in_users,logged_out_users
3,discussiontools,visualeditor,desktop,0.0,2662.0,0.0
7,discussiontools,wikitext,desktop,0.0,139.0,0.0
1,page,visualeditor,desktop,0.0,3777095.0,858968.0
2,page,visualeditor,other,0.0,1317.0,2058.0
4,page,wikitext,desktop,0.0,15163.0,6964.0
5,page,wikitext,phone,0.0,96693.0,204914.0
6,page,wikitext-2017,desktop,0.0,206871.0,21.0
8,page,visualeditor,phone,0.0,2155222.0,1650580.0
9,page,0,phone,0.0,2391609.0,2088736.0
0,0,0,0,0.0,0.0,0.0


Note: We can ignore the high number of '0' or 'null' editor_interface for Mobile Platform. This is related to the bug identified during QA

In [10]:
# Count of all events in visualeditorfeatureuse 

count_vefu_user_edcount = hive.run("""
SELECT
  event.integration AS integration,
  event.editor_interface AS editor_interface,
  event.platform AS platform,
  sum(cast(event.user_editcount=0 as int)) as 0_edits,
  sum(cast(event.user_editcount>0 and event.user_editcount < 100  as int)) as 1to99_edits,
  sum(cast(event.user_editcount>99 as int)) as greater_100_edits

FROM event.visualeditorfeatureuse
WHERE
  year = 2020 
  AND month>=7
  AND wiki <> 'testwiki'
GROUP BY
event.integration, event.editor_interface, event.platform
""")

### User Edit Count check 

In [11]:
count_vefu_user_edcount.sort_values('integration').fillna(0)

,integration,editor_interface,platform,0_edits,1to99_edits,greater_100_edits
2,discussiontools,visualeditor,desktop,1.0,171.0,2490.0
4,discussiontools,wikitext,desktop,0.0,25.0,114.0
1,page,visualeditor,other,2414.0,769.0,192.0
3,page,wikitext,desktop,10137.0,6335.0,5655.0
5,page,visualeditor,desktop,1309229.0,1143732.0,2183102.0
6,page,visualeditor,phone,1920337.0,879622.0,1005843.0
7,page,0,phone,2422331.0,1034256.0,1023758.0
8,page,wikitext,phone,230955.0,36654.0,33998.0
9,page,wikitext-2017,desktop,550.0,10432.0,195910.0
0,0,0,0,0.0,0.0,0.0


Note: We can ignore the high number of '0' or 'null' editor_interface for Mobile Platform. This is related to the bug identified during QA

### Random Sample check

In [12]:
# Count of all events in visualeditorfeatureuse 

count_vefu_random = hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  wiki,
  useragent.browser_family,
  webhost,
  event.integration AS integration,
  event.editor_interface AS editor_interface,
  event.platform AS platform,
  event.bucket AS bucket,
  event.user_id,
  event.feature,
  event.action
FROM event.visualeditorfeatureuse
WHERE
  year = 2020 
  AND date_format(dt, "yyyy-MM-dd")='2020-08-04'
  AND wiki <> 'testwiki'
Limit 10 
""")

In [13]:
count_vefu_random

,date,wiki,browser_family,webhost,integration,editor_interface,platform,bucket,user_id,feature,action
0,2020-08-04,enwiki,Chrome Mobile,en.m.wikipedia.org,page,visualeditor,phone,NaN,0,mwSave,dialog-save
1,2020-08-04,ukwiki,Chrome,uk.wikipedia.org,page,visualeditor,desktop,NaN,521852,clipboard,paste
2,2020-08-04,eswiki,Chrome Mobile,es.m.wikipedia.org,page,visualeditor,phone,NaN,0,link,search-pages-input
3,2020-08-04,srwiki,Chrome Mobile,sr.m.wikipedia.org,page,visualeditor,phone,NaN,0,transclusion,context-show
4,2020-08-04,dewiki,Edge,de.wikipedia.org,page,visualeditor,desktop,NaN,2848033,transclusion,context-show
5,2020-08-04,enwiki,Chrome Mobile,en.m.wikipedia.org,page,visualeditor,phone,NaN,0,mwSave,review-initial-visual
6,2020-08-04,zhwiki,Chrome,zh.wikipedia.org,page,visualeditor,desktop,NaN,2699823,link,search-pages-choose
7,2020-08-04,enwiki,Chrome Mobile,en.m.wikipedia.org,page,visualeditor,phone,NaN,0,transclusion,context-close
8,2020-08-04,jawiki,Mobile Safari,ja.m.wikipedia.org,page,visualeditor,phone,NaN,798737,transclusion,context-show
9,2020-08-04,enwikinews,Chrome Mobile,en.m.wikinews.org,page,visualeditor,phone,NaN,2813202,link,dialog-done


### Are there sessions in VEfu not in EAS ? 

Observed a few sessions in VisualEditorFeatureUse schema (that have null editor_interface) that do not exist in EditAttempStep. 
We are not sure of the exact cause of this, but this may not be an issue as the number of events with null editor_interface is very low from Aug 4, 2020 onwards.

In [ ]:
Vefu_eas_null_edint_session_query = ("""

SELECT distinct vefu.event.editingsessionid AS vefu_editing_session_id,
vefu.event.editor_interface,
vefu.event.integration,
vefu.event.platform

FROM event.visualeditorfeatureuse vefu
WHERE vefu.event.editingsessionid NOT IN
(
SELECT eas.event.editing_session_id
FROM event.editattemptstep eas
WHERE eas.year = 2020 AND eas.month=8 AND eas.day >= 3
)
AND vefu.event.editor_interface is null
AND vefu.year = 2020 AND vefu.month=8 AND vefu.day >= 3
""")

In [15]:
Vefu_eas_null_edint_session = hive.run(
    [
        "SET hive.mapred.mode=nonstrict", 
        Vefu_eas_null_edint_session_query
    ]
)

In [16]:
Vefu_eas_null_edint_session.tail()

,vefu_editing_session_id,editor_interface,integration,platform
1042,fe79d8b167b609b14d8b,NaN,page,phone
1043,fe97b5f1bfa2638d074a,NaN,page,phone
1044,fea6dc0303213101ebd1,NaN,page,phone
1045,febcd7aca52333c9dcf4,NaN,page,phone
1046,ffc59d6581f88f659673,NaN,page,phone


In [33]:
Vefu_eas_null_edint_session_query1 = ("""

SELECT  
--vefu.event.editingsessionid AS vefu_editing_session_id,
vefu.event.editor_interface,
vefu.event.integration,
vefu.event.platform,
count(*) count_sessions
FROM event.visualeditorfeatureuse vefu
WHERE vefu.event.editingsessionid NOT IN
(
SELECT eas.event.editing_session_id
FROM event.editattemptstep eas
WHERE eas.year = 2020 AND eas.month =8 and eas.day > 3 
)
AND vefu.event.editor_interface is null
AND vefu.year = 2020 AND vefu.month = 8 and vefu.day > 3 
GROUP BY vefu.event.editor_interface,
vefu.event.integration,
vefu.event.platform
""")

In [34]:
Vefu_eas_null_edint_session_count = hive.run(
    [
        "SET hive.mapred.mode=nonstrict", 
        Vefu_eas_null_edint_session_query1
    ]
)

In [35]:
Vefu_eas_null_edint_session_count

,editor_interface,integration,platform,count_sessions
0,NaN,NaN,NaN,1
1,NaN,page,phone,29


There are 2023 sessions in VisualEditorFeatureUse that are not present in EditAttemptStep